# V1 test

In [2]:
from trainer.trainer_basic import TSPN_trainer
from configs.config import args
from configs.config import signal_processing_modules,feature_extractor_modules
from pytorch_lightning.loggers import CSVLogger
import torch
import pytorch_lightning as pl

from data.data_provider import get_data
# dataset
train_dataloader, val_dataloader, test_dataloader = get_data(args)

name = f'model:{args.model}_lr:{args.learning_rate}_epochs:{args.num_epochs}_seed:{args.seed}_scale:{args.scale}'

print(f'Running experiment: {name}')
path = 'save/' + name

trainer = pl.Trainer(max_epochs=args.num_epochs,
                      devices= 1,
                      logger=CSVLogger(path, name = 'logs'),
                      log_every_n_steps=1,)

model = TSPN_trainer(signal_processing_modules, feature_extractor_modules, args)
best_model_path = 'save/lr0.001_epochs10_seed17_scale2_l1norm0.01/model-epoch=09-val_loss=0.4744.ckpt'
state_dict = torch.load(best_model_path)
model.load_state_dict(state_dict['state_dict'])

# 使用最佳模型进行测试
trainer.test(model, dataloaders=test_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Running experiment: lr0.001_epochs10_seed17_scale2
# build signal processing layers
# build feature extractor layers
# build classifier
### network:
 Transparent_Signal_Processing_Network(
  (signal_processing_layers): ModuleList(
    (0): SignalProcessingLayer(
      (norm): InstanceNorm1d(2, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (weight_connection): Linear(in_features=2, out_features=6, bias=True)
      (signal_processing_modules): SignalProcessingModuleDict(
        (HT): HilbertTransform()
        (WF): WaveFilters()
        (I): Identity()
      )
      (skip_connection): Linear(in_features=2, out_features=6, bias=True)
    )
    (1-3): 3 x SignalProcessingLayer(
      (norm): InstanceNorm1d(6, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (weight_connection): Linear(in_features=6, out_features=6, bias=True)
      (signal_processing_modules): SignalProcessingModuleDict(
        (HT): HilbertTransform()
        (WF): WaveF

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_acc_epoch       │    0.4578651785850525     │
│      test_loss_epoch      │    1.2863938808441162     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss_epoch': 1.2863938808441162, 'test_acc_epoch': 0.4578651785850525}]